In [1]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

ds = load_dataset('csv', data_files='sinhala_english_sentences8.csv')

In [3]:
import pandas as pd
print(ds)

df = ds["train"].to_pandas()


DatasetDict({
    train: Dataset({
        features: ['english', 'sinhala'],
        num_rows: 84201
    })
})


In [4]:
df.head()

,english,sinhala
0,you will receive a package in the mail,ඔයාට තැපෑලෙන් පැකේජ් එකක් හම්බවේවි
1,you will receive a confirmation code after com...,ලියාපදිංචිය සම්පූර්ණ කලාට පස්සෙ ඔයාට තහවුරු කි...
2,you will receive a discount on your next purchase,ඊලඟ මිලදී ගැනීම කරන කොට ඔයාට වට්ටමක් හම්බවේවි
3,you will receive a phone call from our custome...,ඔයාට අපේ පාරිභෝගික සේවා කණ්ඩායමෙන් දුරකථන ඇමතු...
4,you will receive a notification when your orde...,ඔයාගේ ඇණවුම සූදානම් උනාට පස්සෙ ඔයාට දැනුම් දීම...


In [5]:
df = df.replace('', pd.NA)  # Convert empty strings to NA
df = df.dropna()

In [6]:
df = df[~df.apply(lambda row: row.astype(str).str.contains('#').any(), axis=1)]

In [7]:
df = df[~df.apply(lambda row: row.astype(str).str.contains('%').any(), axis=1)]

In [8]:
has_null = df.isnull().values.any()
print(has_null)  # True if any null exists, False otherwise


False


In [9]:
df.to_csv('output.txt', sep='\t', index=False, encoding="utf-8")

In [10]:
import csv

input_file = r"C:\Users\User\Desktop\sube.txt" 
output_file = r"C:\Users\User\Desktop\yasaru.txt"

pairs = []
eng = ''
sin = ''

# --- Read the source file ----------------------------------------------------
with open(input_file, encoding='utf-8') as f:
    for line in f:
        line = line.strip()

        if line.startswith('English:'):
            eng = line.replace('English:', '').strip()
        elif line.startswith('Sinhala:'):
            sin = line.replace('Sinhala:', '').strip()

            # Only add if both English *and* Sinhala lines have been found
            if eng and sin:
                pairs.append((eng, sin))
                eng = ''
                sin = ''

# --- Write to *plain‑text* (tab‑separated) -----------------------------------
with open(output_file, 'w', encoding='utf-8') as f:
    # Optional header:
    f.write('English\tSinhala\n')

    for eng, sin in pairs:
        f.write(f'{eng}\t{sin}\n')

print(f'✅ Success! Saved as {output_file}')


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User\\Desktop\\sube.txt'

In [ ]:
import csv

# Assuming the input file has English and Sinhala sentences in pairs (e.g., one per line or alternating lines)
# Adjust the input file path and parsing logic based on the actual format
input_file = r"C:\Users\User\Desktop\sube.csv"
output_file = r"C:\Users\User\Desktop\sube1.csv"


pairs = []
with open(input_file, encoding='utf-8') as f:
    eng = ''
    sin = ''
    for line in f:
        line = line.strip()
        if line.startswith('English:'):
            eng = line.replace('English:', '').strip()
        elif line.startswith('Sinhala:'):
            sin = line.replace('Sinhala:', '').strip()
            # Only add if both English and Sinhala exist
            if eng and sin:
                pairs.append((eng, sin))
                eng = ''
                sin = ''

# Write to CSV
with open(output_file, 'w', encoding='utf-8', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(["English", "Sinhala"]) 
    for eng, sin in pairs:
        writer.writerow([eng, sin])

In [ ]:
import pandas as pd

# Step 1: Read the file
with open(r"C:\Users\User\Desktop\MT_System\output.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()


df.to_csv("sinhala_english_sentences8.csv", index=False, encoding="utf-8-sig")

print("CSV file created successfully as sinhala_english_sentences2.csv")


CSV file created successfully as sinhala_english_sentences2.csv


In [11]:
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer
from torch.utils.data import Dataset, DataLoader
from nltk.translate.bleu_score import sentence_bleu
import nltk
from IPython.display import display, HTML



In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/chamodh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
def load_dataset(file_path):
    df = pd.read_csv(file_path)
    display(HTML("<h3>First 5 Rows of Dataset:</h3>" + df.head().to_html()))  # Display dataset preview
    return df['english'].tolist(), df['sinhala'].tolist()

# Load dataset
english_texts, sinhala_texts = load_dataset(r"/Users/chamodh/Documents/CodeSpace/nlp/Sinhala-English-Language-Translator/sinhala_english_sentences8.csv")

,english,sinhala
0,you will receive a package in the mail,ඔයාට තැපෑලෙන් පැකේජ් එකක් හම්බවේවි
1,you will receive a confirmation code after completing the registration,ලියාපදිංචිය සම්පූර්ණ කලාට පස්සෙ ඔයාට තහවුරු කිරීමේ කේතයක් හම්බවේවි
2,you will receive a discount on your next purchase,ඊලඟ මිලදී ගැනීම කරන කොට ඔයාට වට්ටමක් හම්බවේවි
3,you will receive a phone call from our customer service team,ඔයාට අපේ පාරිභෝගික සේවා කණ්ඩායමෙන් දුරකථන ඇමතුමක් හම්බවේවි
4,you will receive a notification when your order is ready for pickup,ඔයාගේ ඇණවුම සූදානම් උනාට පස්සෙ ඔයාට දැනුම් දීමක් හම්බවේවි


In [14]:
class TranslationDataset(Dataset):
    def __init__(self, source_texts, target_texts, tokenizer, max_length=128):
        self.source_texts = source_texts
        self.target_texts = target_texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.source_texts)

    def __getitem__(self, idx):
        source = self.source_texts[idx]
        target = self.target_texts[idx]
        source_enc = self.tokenizer(source, return_tensors="pt", padding="max_length", truncation=True, max_length=self.max_length)
        target_enc = self.tokenizer(target, return_tensors="pt", padding="max_length", truncation=True, max_length=self.max_length)
        return {
            'input_ids': source_enc['input_ids'].squeeze(),
            'attention_mask': source_enc['attention_mask'].squeeze(),
            'labels': target_enc['input_ids'].squeeze()
        }

In [ ]:
%pip install sentencepiece


CondaToSNonInteractiveError: Terms of Service have not been accepted for the following channels. Please accept or remove them before proceeding:
    • https://repo.anaconda.com/pkgs/main
    • https://repo.anaconda.com/pkgs/r

To accept a channel's Terms of Service, run the following and replace `CHANNEL` with the channel name/URL:
    ‣ conda tos accept --override-channels --channel CHANNEL

To remove channels with rejected Terms of Service, run the following and replace `CHANNEL` with the channel name/URL:
    ‣ conda config --remove channels CHANNEL


Note: you may need to restart the kernel to use updated packages.


In [15]:
# Fix the model and tokenizer loading
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Use the same model for both tokenizer and model
model_name = "Helsinki-NLP/opus-mt-en-si"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

model.to(device)
model.eval()

print(f"✅ Model loaded successfully: {model_name}")
print(f"✅ Model is on device: {next(model.parameters()).device}")

Using device: cpu


ImportError: 
MarianTokenizer requires the SentencePiece library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
# Translation function
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

In [ ]:
# BLEU score calculation
def calculate_bleu(reference, hypothesis):
    reference_tokens = [nltk.word_tokenize(reference)]
    hypothesis_tokens = nltk.word_tokenize(hypothesis)
    return sentence_bleu(reference_tokens, hypothesis_tokens, weights=(0.25, 0.25, 0.25, 0.25))

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
pip install torch==2.6.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.16.2 requires torch==2.1.2, but you have torch 2.6.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: C:\Users\User\Desktop\OCR\ocr\Scripts\python.exe -m pip install --upgrade pip



  Obtaining dependency information for torch==2.6.0 from https://files.pythonhosted.org/packages/11/c5/2370d96b31eb1841c3a0883a492c15278a6718ccad61bb6a649c80d1d9eb/torch-2.6.0-cp311-cp311-win_amd64.whl.metadata
  Using cached torch-2.6.0-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Obtaining dependency information for sympy==1.13.1 from https://files.pythonhosted.org/packages/b2/fe/81695a1aa331a842b582453b605175f419fe8540355886031328089d840a/sympy-1.13.1-py3-none-any.whl.metadata
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached torch-2.6.0-cp311-cp311-win_amd64.whl (204.2 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12


In [ ]:
model_name = "Helsinki-NLP/opus-mt-en-mul"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


dataset = TranslationDataset(english_texts, sinhala_texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

ValueError: Due to a serious vulnerability issue in `torch.load`, even with `weights_only=True`, we now require users to upgrade torch to at least v2.6 in order to use the function. This version restriction does not apply when loading files with safetensors.
See the vulnerability report here https://nvd.nist.gov/vuln/detail/CVE-2025-32434

In [ ]:
# Interactive translation testing
def test_translation(input_text):
    translation = translate(input_text)
    # Find reference translation if input matches dataset
    reference = next((sinhala for eng, sinhala in zip(english_texts, sinhala_texts) if eng.lower() == input_text.lower()), translation)
    bleu_score = calculate_bleu(reference, translation)
    return {
        'input': input_text,
        'translation': translation,
        'reference': reference,
        'bleu_score': bleu_score
    }


test_sentences = [
    "I go there with him.",  
    "I need to dance.",  
    "I need to cook.",  
    "I love to travel."  
]

print("\nTesting Translations:\n")
for sentence in test_sentences:
    result = test_translation(sentence)
    display(HTML(f"""
        <div style='margin-bottom: 20px; padding: 10px; border: 1px solid #ccc; border-radius: 5px;'>
            <strong>Input:</strong> {result['input']}<br>
            <strong>Translation:</strong> {result['translation']}<br>
            <strong>Reference:</strong> {result['reference']}<br>
            <strong>BLEU Score:</strong> {result['bleu_score']:.4f}
        </div>
    """))